In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import IPython.display as display
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchmetrics import Precision, Recall

In [ ]:
# train_files_healthy = "Dataset/Train/Train/Healthy"
# train_files_powdery = "Dataset/Train/Train/Powdery"
# train_files_rust = "Dataset/Train/Train/Rust"

# test_files_healthy = "Dataset/Test/Test/Healthy"
# test_files_powdery = "Dataset/Test/Test/Powdery"
# test_files_rust = "Dataset/Test/Test/Rust"

# valid_files_healthy = "Dataset/Validation/Validation/Healthy"
# valid_files_powdery = "Dataset/Validation/Validation/Powdery"
# valid_files_rust = "Dataset/Validation/Validation/Rust"

train_folder_path = '/kaggle/input/plant-leaves-for-image-classification/Plants_2/train'
valid_folder_path = '/kaggle/input/plant-leaves-for-image-classification/Plants_2/valid'
test_folder_path = '/kaggle/input/plant-leaves-for-image-classification/Plants_2/test'

In [ ]:
# for dirname, _, filenames in os.walk(train_folder_path):
#     for filename in filenames:
#         location = os.path.join(dirname, filename)
#         with open(location, 'rb') as f:
#             display.display(display.Image(data=f.read(), width=500))

In [ ]:
# Define transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(45),
    transforms.ToTensor(),
    transforms.Resize((64,64)),
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64,64))
])

valid_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64,64))
])

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(train_folder_path, transform=train_transforms)
valid_dataset = torchvision.datasets.ImageFolder(valid_folder_path, transform=valid_transforms)
test_dataset = torchvision.datasets.ImageFolder(test_folder_path, transform=test_transforms)

In [ ]:
train_dataset

In [ ]:
train_data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_data_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
# train_features, train_labels = next(iter(train_data_loader))
# print(train_features)
# print(train_labels)

In [ ]:
# output shape of conv2d = Width_out = ((Width_in - K + 2P)/S) + 1
# (channels, w, h)

# output shape of maxpool
# (channels, w/kernal_size, h/kernal_size)

In [ ]:
# create helper function to calculate conv2d shape at each step

class Net(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            # [(64-5+(2*1))/1]+1 = 62
            nn.Conv2d(3, 32, kernel_size=5, padding=1), # (3,64,64) -> (32,62,62)
            nn.ELU(),
            # 62/2 = 31
            nn.MaxPool2d(kernel_size=2), # (32,62,62) -> (32,31,31)
            # [32-4+(2*1))/1]+1 = 30
            nn.Conv2d(32, 64, kernel_size=4, padding=1), # (32,32,32) -> (64,29,29)
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2), # (64,30,30) -> (64,15,15)
            nn.Flatten(),
        )
        # Define classifier
        self.classifier = nn.Linear(64*15*15, num_classes)
    
    def forward(self, x):  
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x

In [ ]:
net = Net(num_classes=22)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=.001)

for epoch in range(2):
    for images, labels in train_data_loader:
        optimizer.zero_grad()
        outputs = net(images)
        loss_val = loss(outputs, labels)
        loss_val.backward()
        print('epoch:', epoch)
        print('loss:', loss_val)
        optimizer.step()

In [ ]:
# valid evaluation
precision = Precision(task='multiclass', num_classes=22, average='macro')
recall = Recall(task='multiclass', num_classes=22, average='macro')

net.eval()
with torch.no_grad():
    for images, labels in valid_data_loader:
        outputs = net(images)
        _, preds = torch.max(outputs, 1)
        precision(preds, labels)
        recall(preds, labels)
        
precision = precision.compute()
recall = recall.compute()

In [ ]:
print("Precision:", precision)
print("Recall:", recall)

In [ ]:
from torchmetrics import Accuracy

accuracy = Accuracy(task="multiclass", num_classes=22)

net.eval()
with torch.no_grad():
    for images, labels in valid_data_loader:
        outputs = net(images)
        _, preds = torch.max(outputs, 1)
        accuracy(preds, labels)
        
acc = accuracy.compute()

print(acc)

In [ ]:
from torchmetrics.classification import MulticlassF1Score

f1 = MulticlassF1Score(num_classes=22)

net.eval()
with torch.no_grad():
    for images, labels in valid_data_loader:
        outputs = net(images)
        _, preds = torch.max(outputs, 1)
        f1(preds, labels)
        
f1_score = f1.compute()

print(f1_score)

In [ ]:
f1_manual = (2*precision*recall)/(recall+precision)
f1_manual

In [ ]:
# do any tuning that makes sense

In [ ]:
# retrain 

In [ ]:
# eval on test setfrom torchmetrics.classification import MulticlassF1Score

f1 = MulticlassF1Score(num_classes=22)

net.eval()
with torch.no_grad():
    for images, labels in test_data_loader:
        outputs = net(images)
        _, preds = torch.max(outputs, 1)
        f1(preds, labels)
        
f1_score = f1.compute()

print(f1_score)